In [2]:
import pandas as pd
from geopy.distance import geodesic
from scipy.spatial.distance import cdist
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
# Load the uploaded train sample file
train_sample_file_path = '/content/drive/MyDrive/ML4_adv/최종_위도경도컬럼추가_train.csv'
train_data_sample = pd.read_csv(train_sample_file_path)

# Load the bus feature file
bus_file_path = '/content/drive/MyDrive/ML4_adv/bus_feature.csv'
bus_data = pd.read_csv(bus_file_path)

<ipython-input-3-eb7b6f63eded>:3: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data_sample = pd.read_csv(train_sample_file_path)


### 작업과정

In [ ]:
# Combine '시군구' and '번지' to create a unique identifier, converting '번지' to string
train_data_sample['unique_address'] = train_data_sample['시군구'] + ' ' + train_data_sample['번지'].astype(str)

# Get unique addresses
unique_addresses = train_data_sample['unique_address'].unique()


In [ ]:
# Get unique addresses
unique_addresses = train_data_sample['unique_address'].unique()

# Get latitude and longitude for each unique address with progress indication
lat_lon_dict = {}
total_addresses = len(unique_addresses)
print(f"Total unique addresses: {total_addresses}")

for idx, address in enumerate(unique_addresses):
    subset = train_data_sample[train_data_sample['unique_address'] == address]
    lat = subset['Latitude'].iloc[0]
    lon = subset['Longitude'].iloc[0]
    lat_lon_dict[address] = (lat, lon)

    if (idx + 1) % 10 == 0 or (idx + 1) == total_addresses:
        print(f"Processed {idx + 1}/{total_addresses} addresses")

# Function to calculate the number of bus stops within 500 meters
def calculate_within_500m(lat1, lon1, bus_data):
    count = 0
    for _, row in bus_data.iterrows():
        bus_lat, bus_lon = row['Y좌표'], row['X좌표']
        distance = geodesic((lat1, lon1), (bus_lat, bus_lon)).meters
        if distance <= 500:
            count += 1
    return count

# Calculate the number of bus stops within 500 meters for each unique address with progress indication
address_bus_stop_count = {}
total_lat_lon = len(lat_lon_dict)
print(f"Calculating bus stops within 500m for {total_lat_lon} unique addresses")

for idx, (address, (lat, lon)) in enumerate(lat_lon_dict.items()):
    count = calculate_within_500m(lat, lon, bus_data)
    address_bus_stop_count[address] = count

    if (idx + 1) % 10 == 0 or (idx + 1) == total_lat_lon:
        print(f"Processed {idx + 1}/{total_lat_lon} addresses for bus stop count")

# Add the bus stop count to the original sample data
train_data_sample['500m_내_정류소_수'] = train_data_sample['unique_address'].map(address_bus_stop_count)


Total unique addresses: 8943
Processed 10/8943 addresses
Processed 20/8943 addresses
Processed 30/8943 addresses
Processed 40/8943 addresses
Processed 50/8943 addresses
Processed 60/8943 addresses
Processed 70/8943 addresses
Processed 80/8943 addresses
Processed 90/8943 addresses
Processed 100/8943 addresses
Processed 110/8943 addresses
Processed 120/8943 addresses
Processed 130/8943 addresses
Processed 140/8943 addresses
Processed 150/8943 addresses
Processed 160/8943 addresses
Processed 170/8943 addresses
Processed 180/8943 addresses
Processed 190/8943 addresses
Processed 200/8943 addresses
Processed 210/8943 addresses
Processed 220/8943 addresses
Processed 230/8943 addresses
Processed 240/8943 addresses
Processed 250/8943 addresses
Processed 260/8943 addresses
Processed 270/8943 addresses
Processed 280/8943 addresses
Processed 290/8943 addresses
Processed 300/8943 addresses
Processed 310/8943 addresses
Processed 320/8943 addresses
Processed 330/8943 addresses
Processed 340/8943 addr

KeyboardInterrupt: 

In [ ]:
# Combine '시군구' and '번지' to create a unique identifier, converting '번지' to string
train_data_sample['unique_address'] = train_data_sample['시군구'] + ' ' + train_data_sample['번지'].astype(str)

# Get unique addresses with their latitude and longitude
unique_addresses = train_data_sample[['unique_address', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

# Convert the latitude and longitude data to numpy arrays
bus_coords = bus_data[['Y좌표', 'X좌표']].to_numpy()
address_coords = unique_addresses[['Latitude', 'Longitude']].to_numpy()

# Calculate the distances using cdist
distances = cdist(address_coords, bus_coords, metric='euclidean')

# Calculate the number of bus stops within 500 meters for each unique address
within_500m_count = np.sum(distances <= 0.5, axis=1)

# Map the counts back to the unique addresses
unique_addresses['500m_내_정류소_수'] = within_500m_count

# Merge the counts back to the original data
train_data_sample = train_data_sample.merge(unique_addresses[['unique_address', '500m_내_정류소_수']], on='unique_address', how='left')


In [ ]:
# Save the result to a new CSV file
result_file_path = '/content/drive/MyDrive/ML4_adv/train_data_sample_with_bus_stop_count.csv'
train_data_sample.to_csv(result_file_path, index=False)

In [ ]:
train_data_sample['500m_내_정류소_수'].sample(20)

570295     12584
947739     12584
637067     12584
836136     12584
968170     12584
544347     12584
508005     12584
346021     12584
862433     12584
916474     12584
926067     12584
380141     12584
433362     12584
1070623    12584
777253     12584
638389     12584
931982     12584
123001     12584
875086     12584
469196     12584
Name: 500m_내_정류소_수, dtype: int64

### 최종 작업본

In [ ]:
# Combine '시군구' and '번지' to create a unique identifier, converting '번지' to string
train_data_sample['unique_address'] = train_data_sample['시군구'] + ' ' + train_data_sample['번지'].astype(str)

# Get unique addresses with their latitude and longitude
unique_addresses = train_data_sample[['unique_address', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

# Convert the latitude and longitude data to numpy arrays
bus_coords = bus_data[['Y좌표', 'X좌표']].to_numpy()
address_coords = unique_addresses[['Latitude', 'Longitude']].to_numpy()

# Vectorized haversine function
def haversine(lon1, lat1, lon2, lat2):
    R = 6371.0  # Earth radius in kilometers
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1[:, np.newaxis]
    dlat = lat2 - lat1[:, np.newaxis]
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1)[:, np.newaxis] * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Calculate the distances using haversine formula
distances = haversine(address_coords[:, 1], address_coords[:, 0], bus_coords[:, 1], bus_coords[:, 0])

# Calculate the number of bus stops within 500 meters for each unique address
within_500m_count = np.sum(distances <= 0.5, axis=1)

# Map the counts back to the unique addresses
unique_addresses['500m_내_정류소_수'] = within_500m_count

# Merge the counts back to the original data
train_data_sample = train_data_sample.merge(unique_addresses[['unique_address', '500m_내_정류소_수']], on='unique_address', how='left')



In [ ]:
train_data_sample
train_data_sample.to_csv('/content/drive/MyDrive/ML4_adv/within_500m_bus.csv')

In [ ]:
#train_data_sample.drop(columns=['500m_내_정류소_수_x', '500m_내_정류소_수_y'], inplace=True)

KeyError: "['500m_내_정류소_수_x', '500m_내_정류소_수_y'] not found in axis"

In [ ]:
# Save the result to a new CSV file
result_file_path = '/content/drive/MyDrive/ML4_adv/within500m_buscount.csv'
unique_addresses[['unique_address', '500m_내_정류소_수']].to_csv(result_file_path, index=False)

In [ ]:
train_data_sample

In [ ]:
# Load the original train file
original_train_file_path = '/content/drive/MyDrive/ML4_adv/최종_위도경도컬럼추가_train.csv'
original_train_data = pd.read_csv(original_train_file_path)

# Merge the counts back to the original data
original_train_data['unique_address'] = train_data_sample['시군구'] + ' ' + train_data_sample['번지'].astype(str)
final_data = original_train_data.merge(unique_addresses[['unique_address', '500m_내_정류소_수']], on='unique_address', how='left')



<ipython-input-12-dd8ae0567d2c>:3: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  original_train_data = pd.read_csv(original_train_file_path)


NameError: name 'unique_address' is not defined

In [ ]:
final_data.drop(columns=['unique_address'], inplace=True)
final_data.sample(20)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,단지신청일,target,address,Latitude,Longitude,500m_내_정류소_수
875167,서울특별시 도봉구 창동,821,821.0,0.0,초안산신도브래뉴,80.0192,201309,7,3,2005,...,의무,2015-10-26 14:22:49.0,Y,N,2013-03-07 09:46:20.0,28000,서울특별시 도봉구 창동 821,37.633440,127.042556,26
474080,서울특별시 금천구 시흥동,902-7,902.0,7.0,청우그린,62.6600,201704,12,1,2004,...,NaN,NaN,NaN,NaN,NaN,23400,서울특별시 금천구 시흥동 902-7,37.450520,126.903281,44
998009,서울특별시 양천구 목동,901,901.0,0.0,목동신시가지1,91.2600,200911,25,11,1985,...,NaN,NaN,NaN,NaN,NaN,90000,서울특별시 양천구 목동 901,37.539738,126.877989,26
545487,서울특별시 강남구 개포동,12,12.0,0.0,성원대치2단지,39.5300,201606,27,6,1992,...,NaN,NaN,NaN,NaN,NaN,63000,서울특별시 강남구 개포동 12,37.494197,127.075394,12
836771,서울특별시 영등포구 영등포동,647,647.0,0.0,영등포푸르지오,73.1330,201409,6,9,2002,...,의무,2019-07-10 10:53:45.0,Y,N,2013-03-07 09:46:36.0,42500,서울특별시 영등포구 영등포동 647,37.511678,126.902161,32
138701,서울특별시 강남구 도곡동,91-5,91.0,5.0,삼성래미안,59.9600,200903,19,14,2001,...,NaN,NaN,NaN,NaN,NaN,66500,서울특별시 강남구 도곡동 91-5,37.489818,127.050628,14
560930,서울특별시 강서구 방화동,817,817.0,0.0,동성,76.8000,201511,19,9,1993,...,NaN,NaN,NaN,NaN,NaN,38000,서울특별시 강서구 방화동 817,37.579633,126.812194,12
426931,서울특별시 성북구 하월곡동,222,222.0,0.0,월곡두산위브,59.8550,201901,12,13,2003,...,의무,2016-07-05 10:51:00.0,Y,N,2013-03-07 09:46:42.0,54000,서울특별시 성북구 하월곡동 222,37.606915,127.038492,30
628635,서울특별시 양천구 신정동,325,325.0,0.0,목동신시가지11,66.2400,201604,26,10,1988,...,NaN,NaN,NaN,NaN,NaN,50500,서울특별시 양천구 신정동 325,37.512001,126.858506,23
561190,서울특별시 강서구 방화동,814,814.0,0.0,방화5,39.6000,201607,14,12,1994,...,NaN,NaN,NaN,NaN,NaN,26000,서울특별시 강서구 방화동 814,37.577337,126.810381,20


In [ ]:
# Save the result to a new CSV file
result_file_path = '/content/drive/MyDrive/ML4_adv/작업/final_train_data_with_bus_stop_count.csv'
final_data.to_csv(result_file_path, index=False)